In [44]:
#!/bin/python

import numpy as np
import pandas as pd
from collections import defaultdict
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
import tensorflow as tf
from tensorflow import keras
# from tensorflow.keras import backend as K
# import keras
# from keras import backend as K
# from keras.models import load_model
#import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from pathlib import Path

In [45]:
list_seed = [111,63,682]

prediction_variable = 'ASAS20'

n_nodes = 50
n_layers = 1
lr = '3e-05'
n_batch = 10
n_epochs = 200

std_cutoff = 0.05

df=pd.read_csv('./AS_dataframe_initial_fu_20210208_naiveusers.txt', sep='\t')

if prediction_variable == 'DFT1':
    df = df[df.DFT1 != 2]

if prediction_variable == 'ASAS20':
    df=df[df.ASAS20 != 3]
    
if prediction_variable == 'ASAS40':
    df=df[df.ASAS40 != 3]

colnames = df.columns
result_column_list = ['newID', 'region', 'BSD', 'DFT1', 'DFT2', 'ASDAS1', 'ASDAS2', 'ASAS20', 'ASAS40']
x_colnames_1 = [item for item in colnames if item not in result_column_list]

In [46]:
df_training = df[(df.region!=2) & (df.region != 11) & (df.region != 21) & (df.region != 3) & (df.region != 24)]
#df_independent = df[(df.region == 2) | (df.region == 11) | (df.region == 21) | (df.region == 3) | (df.region == 24)]

In [47]:
# 1) Training dataset

df_training_remov_result = df_training[x_colnames_1] # pre-scaled data
# Remove variables includes only one value.
df_training_remov_novar = df_training_remov_result.loc[:,df_training_remov_result.std() != 0]

# Remove variables includes only small variance.
pre_scaler = preprocessing.MinMaxScaler()
df_training_pre_scaled = pre_scaler.fit_transform(df_training_remov_novar)
remain_boolean = df_training_pre_scaled.std(axis=0) >= std_cutoff
colnames_remain = df_training_remov_novar.columns[remain_boolean]
# colnames_remain will be used for independent data again.

data_x_training_bf_scaling = df_training_remov_novar[colnames_remain].to_numpy()
data_y_training = df_training[prediction_variable].to_numpy()

In [48]:
feature_importances = defaultdict(list)
dict_train = {}
dict_test = {}

seed = list_seed[0]

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)
    
i = 0

for train, test in cv.split(data_x_training_bf_scaling, data_y_training):
    
    dict_train[i] = train
    dict_test[i] = test
    
    i = i + 1

In [54]:

training_scaler = preprocessing.MinMaxScaler()

data_x_training_train = training_scaler.fit_transform(data_x_training_bf_scaling[dict_train[0]])
data_x_training_test = training_scaler.transform(data_x_training_bf_scaling[dict_test[0]])

model = tf.keras.models.load_model('./ANN_ASAS20_50_1_3e-05_111_1.h5')

img_array = tf.convert_to_tensor(data_x_training_train)
last_conv_layer_model = keras.Model(model.inputs, model.output)

with tf.GradientTape() as tape:
    last_conv_layer_output = last_conv_layer_model(img_array)
    tape.watch(last_conv_layer_output)

print(last_conv_layer_output)
print(img_array)
grads = tape.gradient(last_conv_layer_output, img_array)
print(grads)

tf.Tensor(
[[0.67878294]
 [0.69229436]
 [0.7760116 ]
 [0.51582617]
 [0.5941107 ]
 [0.68574923]
 [0.8127461 ]
 [0.7514208 ]
 [0.69298214]
 [0.774083  ]
 [0.6435938 ]
 [0.6407733 ]
 [0.7373961 ]
 [0.82077324]
 [0.67483884]
 [0.6936736 ]
 [0.77810824]
 [0.5690722 ]
 [0.6745034 ]
 [0.56853783]
 [0.70716417]
 [0.815048  ]
 [0.6608246 ]
 [0.7399424 ]
 [0.63365066]
 [0.78460693]
 [0.64714146]
 [0.78861415]
 [0.69726205]
 [0.67867213]
 [0.6360269 ]
 [0.78892004]
 [0.7035413 ]
 [0.5266394 ]
 [0.81089544]
 [0.49124172]
 [0.64064807]
 [0.7280065 ]
 [0.71272546]
 [0.71678895]
 [0.73877645]
 [0.63661623]
 [0.5741825 ]
 [0.6703152 ]
 [0.73625416]
 [0.6823666 ]
 [0.5333111 ]
 [0.7050467 ]
 [0.7098554 ]
 [0.7156087 ]
 [0.6833698 ]
 [0.68079734]
 [0.71286976]
 [0.4973319 ]
 [0.8343092 ]
 [0.7722332 ]
 [0.6477176 ]
 [0.6522367 ]
 [0.83344156]
 [0.42123708]
 [0.6393426 ]
 [0.595451  ]
 [0.61703455]
 [0.81071466]
 [0.65008384]
 [0.66321534]
 [0.80493224]
 [0.6666335 ]
 [0.5928909 ]
 [0.78623676]
 [0.24094